In [1]:
from delta.data import seg_weights_2D
import pathlib
import h5py
import numpy as np
import tifffile

import dask.array as da


In [2]:
!which python

/Users/simonvanvliet/opt/miniconda3/envs/i2i_env/bin/python


In [ ]:
def export_data(path, image, mask, weight):
    for idx, (im, mask, wei) in enumerate(zip(image, mask, weight)):
        im_name = path / 'img' / ('img_%04i' % idx)
        lab_name = path / 'seg' / ('img_%04i' % idx)
        wei_name = path / 'wei' / ('img_%04i' % idx)
        
        tifffile.imwrite(im_name, im)
        tifffile.imwrite(lab_name, mask)
        tifffile.imwrite(wei_name, wei)    
    return None

In [ ]:
root = pathlib.Path(pathlib.Path.home(), 'home', 'Delta2_Caulobacter')
data_dir = root / 'data'

training_file = data_dir /  'training_data_delta.hdf5'

with h5py.File(training_file, 'r') as f:
    chuck_size = (1, 3, *f['mask_train'].shape[-2:])
    mask_train = da.from_array(f['mask_train'], chunks=chuck_size)
    mask_valid = da.from_array(f['mask_valid'], chunks=chuck_size)
    image_train = da.from_array(f['image_train'], chunks=chuck_size)
    image_valid = da.from_array(f['image_valid'], chunks=chuck_size)    


In [ ]:
    
#calculate mask weights
weight_train = da.map_blocks(seg_weights_2D, mask_train)
weight_valid = da.map_blocks(seg_weights_2D, mask_valid)

#force computation
#The seg_weights function of delta is super slow, so this code will take a while to run!
weight_train = weight_train.compute()
weight_valid = weight_valid.compute()


In [ ]:

training_set = data_dir / 'training2'
validation_set = data_dir / 'validation2'
    
training_set.mkdir(exist_ok=True)
validation_set.mkdir(exist_ok=True)

(training_set / 'img').mkdir(exist_ok=True)
(training_set / 'seg').mkdir(exist_ok=True)
(training_set / 'wei').mkdir(exist_ok=True)

(validation_set / 'img').mkdir(exist_ok=True)
(validation_set / 'seg').mkdir(exist_ok=True)
(validation_set / 'wei').mkdir(exist_ok=True)


#export training data to tiff   
export_data(training_set, image_train, mask_train, weight_train)

#export validation data
export_data(validation_set, image_valid, mask_valid, weight_valid)